In [1]:
import geopandas as gpd
import geojsonio as geoio
import pandas as pd
from collections import defaultdict
from shapely.geometry import Point, Polygon
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

# Read Data

In [2]:
vancouver_all_data = gpd.read_file('VancouverAllData.geojson')
vancouver_gdf = gpd.GeoDataFrame({'population':vancouver_all_data['pop'], 'geometry':vancouver_all_data['geometry']})
vancouver_gdf = vancouver_gdf.reset_index()
vancouver_gdf['population'] = pd.to_numeric(vancouver_gdf['population'])

osm_data = pd.read_json('osm/amenities-vancouver.json.gz', lines=True, compression='gzip')
osm_data['geometry'] = list(zip(osm_data['lon'],osm_data['lat']))
osm_data['geometry'] = osm_data['geometry'].apply(Point)
osm_gdf = gpd.GeoDataFrame({'amenity':osm_data['amenity'], 'geometry':osm_data['geometry']}, geometry='geometry', crs=vancouver_gdf.crs)
vancouver_gdf['area'] = vancouver_gdf['geometry'].area
vancouver_gdf['population_density'] = vancouver_gdf['population'] / (vancouver_gdf['area'] * 1000)
vancouver_gdf
# we did this to verify that .explode and .area compute the same area 
# vancouver_gdf_explode = vancouver_gdf.explode('geometry')
# vancouver_gdf_explode
# vancouver_gdf_explode['area'] = vancouver_gdf_explode['geometry'].area
# vancouver_gdf_explode
# vancouver_gdf.max()

/var/folders/dd/2sk2zf7j1wv0jkhkwl_3450h0000gn/T/ipykernel_7134/125131567.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  vancouver_gdf['area'] = vancouver_gdf['geometry'].area


,index,population,geometry,area,population_density
0,0,6154,"MULTIPOLYGON (((-123.02353 49.20818, -123.0235...",0.000126,48910.333782
1,1,8245,"MULTIPOLYGON (((-123.02353 49.20818, -123.0234...",0.000229,35926.018931
2,2,6949,"MULTIPOLYGON (((-123.04245 49.20549, -123.0426...",0.000239,29018.934041
3,3,3908,"MULTIPOLYGON (((-123.05918 49.21162, -123.0597...",0.000097,40318.042652
4,4,4527,"MULTIPOLYGON (((-123.05918 49.21162, -123.0587...",0.000095,47558.899022
...,...,...,...,...,...
484,484,3530,"MULTIPOLYGON (((-122.57911 49.17246, -122.5788...",0.000646,5462.802541
485,485,96,"MULTIPOLYGON (((-122.56953 49.17974, -122.5694...",0.000216,445.127637
486,486,6228,"MULTIPOLYGON (((-122.46134 49.16769, -122.4613...",0.007359,846.345399
487,487,4255,"MULTIPOLYGON (((-122.45998 49.07500, -122.4598...",0.000378,11259.178637


# Data Prep

In [3]:
# use spatial join to match points (osm data) and multipolygons (vancouver data)
pointInPolys = gpd.tools.sjoin(osm_gdf, vancouver_gdf, op="within", how='left')
# index_right is the index of the joined multipolygon, i.e. ID of an area
# group by area and for each area, get the list of amenities located in this area
amenitiesInEachArea = pointInPolys.groupby(['index_right'])['amenity'].apply(list)
# count the number of each amenity in the area and put it into dict
amenitiesInEachArea = amenitiesInEachArea.apply(lambda x: Counter(x))
amenitiesInEachArea
vancouver_gdf['amenities'] = amenitiesInEachArea
# split the dict column into separate columns
vancouver_gdf = pd.concat([vancouver_gdf.drop(['amenities'], axis=1), vancouver_gdf['amenities'].apply(pd.Series)], axis=1)
# some columns are duplicated for some reason??? (removed duplciates based on col names but need to make sure we're not dropping useful data)
vancouver_gdf = vancouver_gdf.loc[:,~vancouver_gdf.columns.duplicated()].drop([0, 'index'], axis=1).fillna(0)

vancouver_gdf

/Users/raufshimarov/opt/anaconda3/envs/geo_env/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3301: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,population,geometry,area,population_density,shelter,post_box,bank,fast_food,restaurant,bench,...,hunting_stand,waste_transfer_station,vacuum_cleaner,lounge,EVSE,storage_rental,atm;bank,healthcare,stripclub,money_transfer
0,6154,"MULTIPOLYGON (((-123.02353 49.20818, -123.0235...",0.000126,48910.333782,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8245,"MULTIPOLYGON (((-123.02353 49.20818, -123.0234...",0.000229,35926.018931,1.0,1.0,5.0,3.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6949,"MULTIPOLYGON (((-123.04245 49.20549, -123.0426...",0.000239,29018.934041,0.0,1.0,0.0,0.0,1.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3908,"MULTIPOLYGON (((-123.05918 49.21162, -123.0597...",0.000097,40318.042652,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4527,"MULTIPOLYGON (((-123.05918 49.21162, -123.0587...",0.000095,47558.899022,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,3530,"MULTIPOLYGON (((-122.57911 49.17246, -122.5788...",0.000646,5462.802541,1.0,0.0,0.0,1.0,12.0,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
485,96,"MULTIPOLYGON (((-122.56953 49.17974, -122.5694...",0.000216,445.127637,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
486,6228,"MULTIPOLYGON (((-122.46134 49.16769, -122.4613...",0.007359,846.345399,0.0,1.0,0.0,3.0,2.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
487,4255,"MULTIPOLYGON (((-122.45998 49.07500, -122.4598...",0.000378,11259.178637,0.0,0.0,0.0,2.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Scale data accroding to the amenities scoring

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA


X = vancouver_gdf.drop(['population', 'geometry', 'area', 'population_density'], axis=1).values
y = vancouver_gdf['population_density'].values # divide by the area of the multipolygon 
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

# train model
model = make_pipeline(
        StandardScaler(),
        PCA(40),
        PolynomialFeatures(degree=1, include_bias=True),
        LinearRegression(fit_intercept=False)
)

model.fit(X_train, y_train)
# validate model
# check r^2 instead of .score
print(model.score(X_valid, y_valid))
predicted_y = model.predict(vancouver_gdf.drop(['population', 'geometry', 'area', 'population_density'], axis=1).values)




-0.06255751561410627


Used predicted y label and the original y label to compute the percentage difference and produce the heatmap based on it


In [5]:
vancouver_gdf['predicted_population_density_difference'] = vancouver_gdf["population_density"] - predicted_y
vancouver_gdf['percentage_difference'] = (vancouver_gdf['predicted_population_density_difference'] / vancouver_gdf['population_density']) * 100

def getColorFromPercentage(percentage):
    if percentage < -80:
        return '#8b7efd'
    elif percentage < -60:
        return '#a89bff'
    elif percentage < -40:
        return '#c3b9ff'
    elif percentage < -20:
        return '#ddd7ff'
    elif percentage < 0:
        return '#e4d4fe'
    elif percentage > 80:
        return '#ff3d61'
    elif percentage > 60:
        return '#ff73a6'
    elif percentage > 40:
        return '#ffa4da'
    elif percentage > 20:
        return '#f4cff8'
    elif percentage >= 0:
        return '#ecd2fc'

vancouver_gdf["amenity_count"] = vancouver_gdf.drop(['population', 'geometry', 'area', 'population_density', 'predicted_population_density_difference',	'percentage_difference'], axis=1).sum(axis=1)
vancouver_gdf['fill'] = vancouver_gdf['percentage_difference'].apply(lambda percentage: getColorFromPercentage(percentage))
heatmap_gdf = gpd.GeoDataFrame({'geometry':vancouver_gdf['geometry'], 'fill':vancouver_gdf['fill'], 'pop_dens': vancouver_gdf['population_density'], 'amenity_count': vancouver_gdf['amenity_count']})
heatmap_gdf
heatmap_gdf.to_file('heatmap.geojson', driver="GeoJSON")
contents = gpd.read_file('heatmap.geojson')
geoio.display(contents)


/Users/raufshimarov/opt/anaconda3/envs/geo_env/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


'http://geojson.io/#id=gist:/3e9f9c4f2b121645c4668a7fc9ef1b38'

In [6]:
red_areas = vancouver_gdf[vancouver_gdf['percentage_difference'] > 0]
blue_areas = vancouver_gdf[vancouver_gdf['percentage_difference'] <= 0]
red_areas


,population,geometry,area,population_density,shelter,post_box,bank,fast_food,restaurant,bench,...,EVSE,storage_rental,atm;bank,healthcare,stripclub,money_transfer,predicted_population_density_difference,percentage_difference,amenity_count,fill
0,6154,"MULTIPOLYGON (((-123.02353 49.20818, -123.0235...",0.000126,48910.333782,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,17453.956354,35.685621,0.0,#f4cff8
3,3908,"MULTIPOLYGON (((-123.05918 49.21162, -123.0597...",0.000097,40318.042652,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,19221.278620,47.674136,4.0,#ffa4da
4,4527,"MULTIPOLYGON (((-123.05918 49.21162, -123.0587...",0.000095,47558.899022,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,19424.788433,40.843646,3.0,#ffa4da
5,3703,"MULTIPOLYGON (((-123.07203 49.21459, -123.0727...",0.000067,55275.867375,0.0,0.0,0.0,1.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,29722.076182,53.770438,6.0,#ffa4da
8,5548,"MULTIPOLYGON (((-123.09530 49.20528, -123.0954...",0.000145,38130.801214,0.0,0.0,0.0,1.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9536.181193,25.009129,5.0,#f4cff8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,4373,"MULTIPOLYGON (((-122.85647 49.19894, -122.8564...",0.000174,25168.677134,0.0,3.0,0.0,0.0,0.0,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3072.789167,12.208783,31.0,#ecd2fc
451,2917,"MULTIPOLYGON (((-122.84531 49.20618, -122.8453...",0.000062,47233.064422,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,14556.059649,30.817521,2.0,#f4cff8
466,5317,"MULTIPOLYGON (((-122.65995 49.09549, -122.6601...",0.000072,73565.945990,0.0,0.0,0.0,1.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,50074.925645,68.068078,21.0,#ff73a6
467,5125,"MULTIPOLYGON (((-122.64681 49.10222, -122.6468...",0.000104,49215.154677,0.0,2.0,0.0,5.0,13.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,624.299512,1.268511,43.0,#ecd2fc


Now we find what amenities are important in red areas and in blue areas

In [7]:
red_areas_amenities = red_areas.drop(['population', 'geometry', 'area', 'population_density', 'predicted_population_density_difference',	'percentage_difference', 'amenity_count'], axis=1)
red_areas_amenities_avg = red_areas_amenities.mean().sort_values(ascending=False)
# red_areas_amenities_avg = red_areas_amenities_avg[red_areas_amenities_avg != 0]

blue_areas_amenities = blue_areas.drop(['population', 'geometry', 'area', 'population_density', 'predicted_population_density_difference',	'percentage_difference', 'amenity_count'], axis=1)
blue_areas_amenities_avg = blue_areas_amenities.mean().sort_values(ascending=False)
# blue_areas_amenities_avg = blue_areas_amenities_avg[blue_areas_amenities_avg != 0]

# red_areas_amenities_avg = red_areas_amenities_avg.to_frame().rename({0: 'red_avg'}, axis=1)
# blue_areas_amenities_avg = blue_areas_amenities_avg.to_frame().rename({0: 'red_avg'}, axis=1)
red_and_blue_avg = pd.concat([red_areas_amenities_avg, blue_areas_amenities_avg], axis=1).rename({0: 'red_avg', 1:'blue_avg'}, axis=1)
# red_and_blue_avg = red_and_blue_avg[red_and_blue_avg['red_avg'] < red_and_blue_avg['blue_avg']]
red_and_blue_avg


/var/folders/dd/2sk2zf7j1wv0jkhkwl_3450h0000gn/T/ipykernel_7134/2496838260.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  red_areas_amenities_avg = red_areas_amenities.mean().sort_values(ascending=False)
/var/folders/dd/2sk2zf7j1wv0jkhkwl_3450h0000gn/T/ipykernel_7134/2496838260.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  blue_areas_amenities_avg = blue_areas_amenities.mean().sort_values(ascending=False)


,red_avg,blue_avg
bench,5.994819,6.564189
restaurant,4.248705,4.783784
bicycle_parking,2.590674,2.868243
cafe,1.792746,2.091216
fast_food,1.569948,2.219595
...,...,...
letter_box,0.000000,0.003378
smoking_area,0.000000,0.003378
lobby,0.000000,0.003378
safety,0.000000,0.003378


we did the T-test on whether the avg num of amenities in red differ from the avg num of amenities in blue since the var is ar close

we then confirmed the result using mannwhitneyu

In [8]:
red_and_blue_avg = red_and_blue_avg.dropna()

from scipy import stats
print(stats.normaltest(red_and_blue_avg['red_avg']).pvalue)
print(stats.levene(red_and_blue_avg['red_avg'], red_and_blue_avg['blue_avg']).pvalue)

ttest = stats.ttest_ind(red_and_blue_avg['red_avg'], red_and_blue_avg['blue_avg'])
print(ttest)
print(ttest.statistic)
print(ttest.pvalue)

print(stats.mannwhitneyu(red_and_blue_avg['red_avg'], red_and_blue_avg['blue_avg']).pvalue)


8.805475187142767e-39
0.7584100858996414
Ttest_indResult(statistic=-0.30413844757344716, pvalue=0.76128377053823)
-0.30413844757344716
0.76128377053823
0.9702150240083519


Since p-value is high, we failed to reject H0 (which is red= blue), and we conclude that the avg num of amenities in red differ from the avg num of amenities in blue

number of amenities and population density of the area are slightly positvely correlated

In [34]:
red_areas_amenities[red_areas_amenities.isnull().any(axis=1)]

,shelter,post_box,bank,fast_food,restaurant,bench,fuel,library,cafe,post_office,...,hunting_stand,waste_transfer_station,vacuum_cleaner,lounge,EVSE,storage_rental,atm;bank,healthcare,stripclub,money_transfer


In [9]:
red_areas['amenity_count'].corr(red_areas['population_density'])

0.43022836230919903

In [44]:
# red_areas_amenities = red_areas.drop(['population', 'geometry', 'area', 'population_density', 'predicted_population_density_difference',	'percentage_difference', 'amenity_count', 'fill'], axis=1)
# red_areas
# for col in red_areas:
#     print(red_areas[col].corr(red_areas['population_density']))
# red_areas_amenities = red_areas_amenities.drop(['fill'], axis=1)
correlations = {}
for col in red_areas_amenities:
    # print(red_areas_amenities[col], red_areas['population_density'])
    correlations[col] = red_areas_amenities[col].corr(red_areas['population_density'])
correlations = dict(sorted(correlations.items(), key=lambda item: abs(item[1]), reverse=True))
correlations


{'motorcycle_rental': nan,
 'Pharmacy': nan,
 'car_sharing': 0.6450369048782993,
 'bicycle_rental': 0.5949052967061216,
 'restaurant': 0.45210321596704733,
 'cafe': 0.4293151101061623,
 'bicycle_parking': 0.4083281072524948,
 'post_box': 0.3972864416090718,
 'bar': 0.31395796351542743,
 'pub': 0.30919537709649864,
 'bench': 0.3008341397812216,
 'fast_food': 0.2667825446535662,
 'cinema': 0.26610442686123564,
 'waste_basket': 0.2636311956866861,
 'car_rental': 0.2609077007659062,
 'pharmacy': 0.2428645851633142,
 'post_office': 0.22785928686871343,
 'bank': 0.21574791001439167,
 'fuel': -0.21404805255620302,
 'fountain': 0.16228854772357537,
 'ice_cream': 0.1546049885898398,
 'community_centre': 0.15434641185195955,
 'shelter': -0.1514588160017624,
 'clinic': 0.14782844202132103,
 'theatre': 0.1401242181923878,
 'toilets': -0.13382492902532142,
 'public_bookcase': -0.11697147599386933,
 'dentist': 0.10772844094908761,
 'telephone': -0.10584514582131674,
 'veterinary': -0.101480894204770

Car sharing is sur

In [11]:
red_areas_amenities

,shelter,post_box,bank,fast_food,restaurant,bench,fuel,library,cafe,post_office,...,waste_transfer_station,vacuum_cleaner,lounge,EVSE,storage_rental,atm;bank,healthcare,stripclub,money_transfer,fill
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,#f4cff8
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,#ffa4da
4,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,#ffa4da
5,0.0,0.0,0.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,#ffa4da
8,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,#f4cff8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,0.0,3.0,0.0,0.0,0.0,17.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,#ecd2fc
451,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,#f4cff8
466,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,#ff73a6
467,0.0,2.0,0.0,5.0,13.0,0.0,0.0,1.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,#ecd2fc


In [ ]:

# import json

# with open("amenities_scores.txt") as f:
#     data = f.read()
# scores = json.loads(data)

# df_train = vancouver_gdf.drop(['population', 'area', 'population_density', 'geometry'], axis=1)
# # df_train += 1
# scores_vector = []
# for i in range(len(df_train.columns)):
#     scores_vector.append(scores[df_train.columns[i]])
# scores_vector

# scaled_df_train = df_train * scores_vector
# scaled_df_train["sum"] = scaled_df_train.sum(axis=1)
# scaled_df_train['population_density'] = vancouver_gdf['population_density']
# # scaled_df_train = scaled_df_train[(scaled_df_train['sum'] < 1000) & (scaled_df_train['sum'] > 50)]
# scaled_df_train
# # plt.scatter(scaled_df_train['sum'], scaled_df_train['population_density'], 0.5)





# Machine Learning

Our x is the number of amenities of some type in the area (e.g. 2 cafes, 4 banks, etc.). Our y is the population density in this area.<br>
TO DO: Did we want to manually calculate a score for each area and use it as training data?

In [ ]:
# from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.preprocessing import StandardScaler
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.neural_network import MLPRegressor
# import matplotlib.pyplot as plt

# train_data = vancouver_gdf.drop(['population', 'area', 'population_density', 'geometry'], axis=1)
# valid_data = vancouver_gdf['population', 'area', 'population_density', 'geometry']
# train_data
# X = scaled_df_train["sum"].values
# y = scaled_df_train['population_density'].values # divide by the area of the multipolygon 
# X_train, X_valid, y_train, y_valid = train_test_split(X, y)

# X = np.stack([X], axis=1)
# model = LinearRegression(fit_intercept=True)
# model.fit(X, y)
# predicted_population = model.intercept_ + model.coef_[0] * X
# plt.scatter(scaled_df_train["sum"], scaled_df_train['population_density'], s=0.5)
# plt.plot(X, predicted_population, '-r')
# predicted_population = np.moveaxis(predicted_population, 1, 0)

# # scaled_df_train['predicted_population_density_difference'] = scaled_df_train["population_density"] - predicted_population[0]
# # scaled_df_train

# # plt.plot(X, predicted_population, '-r')

# # loook at r^2 to see how useful our model is

# # train model
# model = make_pipeline(
#     StandardScaler(),
#     PolynomialFeatures(degree=5, include_bias=True),
#     LinearRegression(fit_intercept=False)
    # MLPRegressor(hidden_layer_sizes=(100, 20),
    #                  activation='logistic', solver='lbfgs')
# )

# model.fit(np.stack([X_train], axis=1), np.stack([y_train], axis=1))
# # validate model
# # check r^2 instead of .score
# print(model.score(np.stack([X_valid], axis=1), np.stack([y_valid], axis=1)))

# 

In [ ]:
X2 = scaled_df_train['population_density'].values # divide by the area of the multipolygon 
y2 = scaled_df_train[['shelter', 'post_box', 'bank']].values # TODO: include other amenities
X2_train, X2_valid, y2_train, y2_valid = train_test_split(X2, y2)

# train model
model = make_pipeline(
    StandardScaler(),
    PolynomialFeatures(degree=5, include_bias=True),
    LinearRegression(fit_intercept=False)
    # MLPRegressor(hidden_layer_sizes=(100, 20),
    #                  activation='logistic', solver='lbfgs')
)

model.fit(np.stack([X2_train], axis=1), y2_train)
model.predict
# # validate model
# # check r^2 instead of .score
print(model.score(np.stack([X2_valid], axis=1), y2_valid))


NameError: name 'scaled_df_train' is not defined

# Heatmap

Red: overpopulated<br>
Yellow: OK<br>
Green: underpopulated<br>

if we take into account our idea of counting the amenities and predicting the population, the population distribuiton would look like the following:

In [ ]:
vancouver_gdf['predicted_population_density_difference'] = scaled_df_train["population_density"] - predicted_population[0]
vancouver_gdf['percentage_difference'] = (vancouver_gdf['predicted_population_density_difference'] / vancouver_gdf['population_density']) * 100
vancouver_gdf

def getColorFromPercentage(percentage):
    if percentage < -90:
        return '#1034A6'
    elif percentage < -45:
        return '#412F88'
    elif percentage < 0:
        return '#722B6A'
    elif percentage > 90:
        return '#A2264B'
    elif percentage > 45:
        return '#D3212D'
    elif percentage >= 0:
        return '#F62D2D'


vancouver_gdf['fill'] = vancouver_gdf['percentage_difference'].apply(lambda percentage: getColorFromPercentage(percentage))
heatmap_gdf = gpd.GeoDataFrame({'geometry':vancouver_gdf['geometry'], 'fill':vancouver_gdf['fill']})
heatmap_gdf

heatmap_gdf.to_file('heatmap.geojson', driver="GeoJSON")

/Users/raufshimarov/opt/anaconda3/envs/geo_env/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


# Analysis

since our model sucks, we decided to do t-test and confirm that the two vars are not related

TO DO: among the red areas, which amenity is the most important?

In [ ]:
from scipy import stats
# ttest = stats.ttest_ind(scaled_df_train["sum"], scaled_df_train['population_density'])
# ttest

corr = scaled_df_train["sum"].corr(scaled_df_train['population_density'])
corr

# check normality
print(stats.normaltest(scaled_df_train['population_density']).pvalue)
print(stats.normaltest(scaled_df_train["sum"]).pvalue)
# not normal, so we use non-param tests

# high sum

scaled_df_train_median_sum = scaled_df_train['sum'].median()
scaled_df_train_high_sums = scaled_df_train[scaled_df_train['sum'] > scaled_df_train_median_sum]
scaled_df_train_low_sums = scaled_df_train[scaled_df_train['sum'] <= scaled_df_train_median_sum]

print(stats.mannwhitneyu(scaled_df_train_high_sums['population_density'], scaled_df_train_low_sums['population_density']))

4.690974170543989e-73
4.3119286584947694e-126
MannwhitneyuResult(statistic=35704.0, pvalue=0.00019793370342218784)
